In [187]:
import pandas as pd
import numpy as np
import joblib

In [188]:
data = pd.read_csv('datalabel.csv')

In [189]:
data.shape

(1794, 3)

In [190]:
data.head(5)

,Date,Content,Sentiment
0,2022-04-08 02:53:31+00:00,"b'@MythiaVTuber gorengan wkwk bihun mit, dimsu...",Neutral
1,2022-04-08 02:47:43+00:00,b'@LeifaNerine @MythiaVTuber SeBuT n@m4 KU k@k...,Positive
2,2022-04-08 02:46:05+00:00,b'@LeifaNerine @MythiaVTuber Mantap sekali ipe...,Positive
3,2022-04-08 02:40:35+00:00,b'@simper_pro @MythiaVTuber tak gemplang kau yak',Negative
4,2022-04-08 02:35:53+00:00,b'Malam ini mau ngobrolin etika berkunjung ke ...,Neutral


In [191]:
data.tail(5)

,Date,Content,Sentiment
1789,2022-04-03 07:14:14+00:00,b'@airaniiofifteen Pagi yopi senpai~',Positive
1790,2022-04-03 06:34:32+00:00,b'@airaniiofifteen Itu tidur apa makan beb? sa...,Positive
1791,2022-04-03 06:31:01+00:00,b'@airaniiofifteen Pagi yopi ~',Positive
1792,2022-04-03 06:12:06+00:00,b'@airaniiofifteen Pagi Airaniiii',Positive
1793,2022-04-03 06:02:50+00:00,b'@airaniiofifteen Waduh obat flu apaan sampe ...,Positive


In [192]:
data.sample(5)

,Date,Content,Sentiment
1623,2022-04-04 02:41:22+00:00,b'@airaniiofifteen Pagi Yopi!',Positive
237,2022-04-03 21:55:57+00:00,"b'@MythiaVTuber remember, just having fun, don...",Positive
1636,2022-04-04 02:34:18+00:00,b'@airaniiofifteen pagi!!!!!!!!\xf0\x9f\x8c\x8...,Positive
1100,2022-04-06 03:00:13+00:00,b'@airaniiofifteen Pagi Iofi \xf0\x9f\x92\x9c\...,Positive
1508,2022-04-04 16:20:38+00:00,b'@airaniiofifteen Kawaiiii https://t.co/rktyK...,Positive


In [193]:
data.describe()

,Date,Content,Sentiment
count,1794,1794,1793
unique,1749,1564,5
top,2022-04-06 03:02:53+00:00,b'@airaniiofifteen Pagi!',Positive
freq,3,31,1511


In [194]:
data.columns

Index(['Date', 'Content', 'Sentiment'], dtype='object')

In [196]:
data['Sentiment'].value_counts()

Positive    1511
Neutral      195
Negative      85
P'P            1
positive       1
Name: Sentiment, dtype: int64

In [197]:
data['Sentiment'] = data['Sentiment'].map({'Neutral':'pass', 'Negative':'gak ok', 'Positive':'ok'})

In [199]:
data['Sentiment'].value_counts()

ok        1511
pass       195
gak ok      85
Name: Sentiment, dtype: int64

Train_Test

In [200]:
from sklearn.model_selection import train_test_split

In [201]:
X=data.Content
y=data.Content

In [202]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=225)

In [203]:
print('Banyak data x_train :',len(x_train))
print('Banyak data x_test  :',len(x_test))
print('Banyak data y_train :',len(y_train))
print('Banyak data y_test  :',len(y_test))

Banyak data x_train : 1614
Banyak data x_test  : 180
Banyak data y_train : 1614
Banyak data y_test  : 180


Text Preprocessing

In [205]:
import re

from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer


tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9_]+' #menghilangkat username jika twitter
pat2 = r'https?://[^ ]+' #menghilangkan situs website
combined_pat = r'|'.join((pat1, pat2)) #join pat1 dan pat 2
www_pat = r'www.[^ ]+' #menhilangkan situs website

#set_stopword yang di deskripsikan sendiri
stopword_user = set(pd.read_fwf('stopwords.txt', sep='\n', header=0))

In [206]:
def proses_teks(teks):
    soup = BeautifulSoup(teks, 'lxml')
    souped = soup.get_text()
    try:
        teks = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        teks = souped
    teks_bersih= re.sub("[^a-zA-Z0-9]", " ",(re.sub(www_pat, '', re.sub(combined_pat, '', teks)).lower()))
    teks_bersih= ' '.join([word for word in teks_bersih.split() if word not in stopword_user])
    return (" ".join([x for x in tok.tokenize(teks_bersih) if len(x) > 1])).strip()

In [19]:
teks = '''Pemerintah selalu konsisten dalam menjaga kelestarian lingkungan hidup. Jokowi Bangun Nusantara https://t.co/XwsCkHx8e3'''

In [20]:
proses_teks(teks)

'pemerintah selalu konsisten dalam menjaga kelestarian lingkungan hidup jokowi bangun nusantara'

In [208]:
x=[]
for teks in data.Content:
    x.append(proses_teks(teks))

In [209]:
clean_text=pd.DataFrame({'clean_text':x})

In [210]:
data=pd.concat([data,clean_text],axis=1)

In [211]:
data.head(5)

,Date,Content,Sentiment,clean_text
0,2022-04-08 02:53:31+00:00,"b'@MythiaVTuber gorengan wkwk bihun mit, dimsu...",pass,gorengan wkwk bihun mit dimsum cendol es buah ...
1,2022-04-08 02:47:43+00:00,b'@LeifaNerine @MythiaVTuber SeBuT n@m4 KU k@k...,ok,sebut ku b1ar m4m4hku
2,2022-04-08 02:46:05+00:00,b'@LeifaNerine @MythiaVTuber Mantap sekali ipe...,ok,mantap sekali ipeeehhhh
3,2022-04-08 02:40:35+00:00,b'@simper_pro @MythiaVTuber tak gemplang kau yak',gak ok,tak gemplang kau yak
4,2022-04-08 02:35:53+00:00,b'Malam ini mau ngobrolin etika berkunjung ke ...,pass,malam ini mau ngobrolin etika berkunjung ke li...


Vectorizer

In [212]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, HashingVectorizer

In [213]:
cvec=CountVectorizer()
tvec=TfidfVectorizer()
hvec=HashingVectorizer()

Model Definition

In [214]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

In [215]:
clf1 = RandomForestClassifier()
clf2 = LogisticRegression()
clf3 = BernoulliNB()
clf4 = SVC()

In [216]:
from sklearn.pipeline import Pipeline

In [240]:
model= Pipeline([('vectorizer',tvec)
                 ,('classifier',clf1)])

In [241]:
model.fit(x_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', RandomForestClassifier())])

In [236]:
hasil=model.predict(x_test)

Check

In [237]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [238]:
confusion_matrix(hasil,y_test)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [239]:
accuracy_score(hasil,y_test)

0.011111111111111112

Save Model

In [224]:
# save the model to disk
import pickle

filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

load model

In [225]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)